<a href="https://colab.research.google.com/github/jeff-ai-ml/genai/blob/main/Simple_ChromaDB_Notebook_for_News_Article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.6 MB/s e

In [3]:
# 🧠 Import libraries
import chromadb
from sentence_transformers import SentenceTransformer
from chromadb.utils import embedding_functions

In [4]:
# 🔧 Initialize ChromaDB client (in-memory)
client = chromadb.Client()

In [5]:
# 🏷️ Create a collection to store news articles
collection = client.create_collection(name="news_articles")

In [6]:
# 🔡 Use sentence-transformers to embed texts
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# ⏬ Custom embedding function for Chroma
class MyEmbeddingFunction(embedding_functions.EmbeddingFunction):
    def __call__(self, texts):
        return embed_model.encode(texts).tolist()

In [9]:
# 🔁 Re-create collection with embedding function
client.delete_collection("news_articles")
collection = client.create_collection(name="news_articles", embedding_function=MyEmbeddingFunction())


/tmp/ipython-input-9-1862985356.py:3: DeprecationWarning: The class MyEmbeddingFunction does not implement __init__. This will be required in a future version.
  collection = client.create_collection(name="news_articles", embedding_function=MyEmbeddingFunction())


In [10]:
# ✍️ Step 1: Accept news articles from user
news_list = [
    "The stock market surged today with tech stocks leading the rally.",
    "India won the cricket match against Australia by 6 wickets.",
    "NASA announced a new moon mission planned for 2026.",
    "Heavy rainfall is expected in Mumbai over the weekend.",
    "The government has launched a new education policy reform.",
]


In [11]:
# 🗂️ Add news articles to ChromaDB
collection.add(
    documents=news_list,
    ids=[f"news-{i}" for i in range(len(news_list))]
)

In [12]:
print("✅ News articles added to ChromaDB.\n")

✅ News articles added to ChromaDB.



In [24]:
# ❓ Step 2: Ask a user query
query = input("Ask your question (e.g., 'Tell me about the weather in Mumbai'): ")


Ask your question (e.g., 'Tell me about the weather in Mumbai'): mars


In [25]:
# 🔍 Step 3: Search relevant article
results = collection.query(
    query_texts=[query],
    n_results=1  # return top 1 match
)

In [26]:
# 🧾 Step 4: Show result
if results['documents']:
    print("\n📄 Most relevant article found:")
    print(results['documents'][0][0])
else:
    print("No matching article found.")


📄 Most relevant article found:
NASA announced a new moon mission planned for 2026.
